<a href="https://colab.research.google.com/github/Chiaradisanto/Segmentation/blob/main/Copia_di_FinalCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  **Save DICOM FILES as .png images**



Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')


Importing libraries


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data analysis and manipulation
import os #operating systems
import scipy # multidimensional image processing
from matplotlib import pyplot as plt # plot visualization
from  scipy import ndimage



Installing and importing module for reading and writing DICOM files

In [ ]:
!pip install pydicom
import pydicom
from pydicom import dcmread


**get_names** function takes as input the path of the selected subject containing the DICOM files 



In [ ]:
def get_names(path):
    names = []
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            _, ext = os.path.splitext(filename)
            if ext in ['.dcm']:
                names.append(filename)
    return names

DICOM files are ordered with sort function. 

In [ ]:
names = sorted(get_names(path)) # path of the .dicom files

A list containig all the DICOM files is created. 
This operation loads all DICOM files from the folder into a list for manipulation.

In [ ]:
files = []
for x in names:
        files.append(pydicom.dcmread(path+'/'+x))
len(files)


Make sure that all the files contains SliceLocation in it. Otherwhise skip them



In [ ]:
slices = []
skipcount = 0
for f in files:
    if hasattr(f, 'SliceLocation'):
        slices.append(f)
    else:
        skipcount = skipcount + 1

print("skipped, no SliceLocation: {}".format(skipcount))

Slices are sorted according to SliceLocation

In [ ]:
slices = sorted(slices, key=lambda s: s.SliceLocation, reverse=True)  #reverse is True to sort in descending order


The voxel values in the images are raw. 

**dicom_HU** converts raw values into Houndsfeld units.
This function takes as input all the slices of the considered subject.
The transformation is linear. 

Both the rescale intercept and rescale slope are stored in the DICOM headers at the time of image acquisition.
The final value is rescaled to HU.
Windowing is then applied in order to adjust the grayscale level of the images.




In [ ]:
def dicom_HU(scan):
    image = np.stack([s.pixel_array*s.RescaleSlope+s.RescaleIntercept for s in scan],axis=2).astype(np.int16) 

    img_min = 120 - 120 // 2 #minimum HU level
    img_max = 120 + 120 // 2 #maximum HU level
    window_image = image.copy()

    window_image[window_image < img_min] = img_min #set img_min for all HU levels less than minimum HU level
    window_image[window_image > img_max] = img_max #set img_max for all HU levels higher than maximum HU level
    plt.figure(figsize=(20, 10))
  
    plt.style.use('grayscale') 
    plt.imshow(window_image[:,:,0], cmap='gray')
    plt.axis('off')
    return  np.array(window_image)

Save as images in .png format in a specific folder. A folder for each patient is created.

In [ ]:
num_slices=len(files)
for i in range(1,num_slices):
    scan=dicom_HU(slices[i-1:i])
    plt.axis('off')
    plt.savefig(f"{images_path}"+str(i)+".png", bbox_inches='tight',pad_inches = 0,dpi=300, quality=95) #dpi represents the resolution in dots per inch.
                                                                                                        # A high value results in a high resolution image.
                                                                                                    
                                                                                                        #quality represents the image quality
                                                                                                        #on a scale from 1 (worst) to 95 (best)
    plt.show()

These operations must be repeated for all the subjects changing the paths.

### Save **ROI.nrrd** files as png. masks

Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Installing SimpleITK to read .nrrd files 

In [ ]:
!pip install SimpleITK


Importing libraries


In [ ]:
import glob #to find files
import os
import SimpleITK as sitk
import nibabel as nib
from matplotlib import pyplot as plt
import numpy as np


In [ ]:
ROI_path= glob.glob('/gdrive/MyDrive/ROI_out.nrrd') # Path of the .nrrd file


Read the .nrrd files

In [ ]:
mask_sitk  = sitk.ReadImage(ROI_path[0]) #read the masks
sitk_shape=mask_sitk.GetSize() #get the masks size
print(sitk_shape) # the first 2 dimensions are height and width , the last dimension represents the number of masks in the folder

Save as masks in .png format in a  specific folder.
A folder for each subject is created.
*images path* is the final folder to store the masks 

In [ ]:
num_slices=sitk_shape[2]
for i in range(1,num_slices):
    img=plt.imshow(sitk.GetArrayViewFromImage(mask_sitk)[i], cmap='gray')    
    plt.axis('off')
    plt.savefig(f"{images_path}"+str(i)+".png", bbox_inches='tight',pad_inches = 0,dpi=300, quality=95) #dpi represents the resolution in dots per inch.
                                                                                                        # A high value results in a high resolution image.
                                                                                                    
                                                                                                        #quality represents the image quality
                                                                                                        #on a scale from 1 (worst) to 95 (best)
    plt.show()

These operations must be repeated for all the subjects changing the paths.

21 folders are created, one for each subject. At the end of these steps, each folder contains two subfolders "images" and "masks" with the associated images and masks for each subject.
Subjects's folders are splitted in TRAINING and VALIDATION folders.
18 subjects are in TRAINING folder while 3 subjects are in VALIDATION one.
7-fold cross validation is then performed.

### Final folders creation

Once the images and masks of each subject have been saved , in the next part of the code four folders have been created: 'train_images' and 'train_masks' in which are all the images and the associated masks of the training set, and 'validation_images' and 'validation_masks' which contains the images and the associated masks of the validation set. This operation is necessary to perform Data Augumentation in a correct way, described in the later steps of the code.
The images and the associated masks are charaterized by the same name.

Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Importing libraries

In [ ]:
import os
import numpy as np
from PIL import Image 
import cv2 
from glob import glob
from tqdm import tqdm
from natsort import natsorted
from google.colab.patches import cv2_imshow


In [ ]:
TRAIN_path='/gdrive/MyDrive/TRAIN' #training path
VALIDATION_path='/gdrive/MyDrive/VALIDATION'# validation path

Loading the images and masks of training set

In [ ]:
def load_train_data(train_path):

    train_x = (glob(f"{TRAIN_path}/*/images/*.png"))
    train_y = (glob(f"{TRAIN_path}/*/masks/*.png"))
 
    return train_x,train_y

In [ ]:
(train_x,train_y) = load_train_data(TRAIN_path)


Images and Masks of the training set are sorted naturally with natsorted function 

In [ ]:
train_x=natsorted(train_x)
train_y=natsorted(train_y)
print(len(train_x))   #train_x and train_y must have the same length , since they must contains the same number of files.
print(len(train_y))

Loading the images and masks of validation set


In [ ]:
def load_val_data(val_path):
    val_x= (glob(f"{VALIDATION_path}/*/images/*.png"))
    val_y= (glob(f"{VALIDATION_path}/*/masks/*.png"))
    return val_x,val_y

In [ ]:
(val_x,val_y) = load_val_data(VALIDATION_path)


Images and Masks of the validation set are sorted naturally with natsorted function.


In [ ]:
val_x=natsorted(val_x)
val_y=natsorted(val_y)
print(len(val_x))
print(len(val_y))

Creating the 'train_images' and 'train_masks' folders

In [ ]:
image_path_train='/gdrive/MyDrive/train_images/images/' # final images path
mask_path_train='/gdrive/MyDrive/train/train_masks/masks/' #final masks path

In [ ]:
H = 256 #height 
W = 256 #width
for idx, (x, y) in tqdm(enumerate(zip(train_x, train_y)), total=len(train_x)):
        """ Extracting the folder name and image name for each subject """
        dir_name = x.split("/")[-3]
        name = dir_name + "_" + x.split("/")[-1].split(".")[0] #extract the name of every subject's folder 
        """ Read the image and mask """
        x = cv2.imread(x, cv2.IMREAD_GRAYSCALE) #read the image
        y = cv2.imread(y, cv2.IMREAD_GRAYSCALE) #read the mask
        X = [x]
        Y = [y]
        idx = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H),interpolation = cv2.INTER_CUBIC)   #images are resized from 512x512 to 256x256 through bicubic interpolation
            m = cv2.resize(m, (W, H),interpolation = cv2.INTER_CUBIC)   #masks are resized from 512x512 to 256x256 through bicubic interpolation

            tmp_image_name = f"{name}.png"  #image and mask are saved with the same name 
            tmp_mask_name  = f"{name}.png" 

            image_path = os.path.join(image_path_train, tmp_image_name)
            mask_path  = os.path.join(mask_path_train, tmp_mask_name)
            print(tmp_image_name)

            cv2.imwrite(image_path,i, [int(cv2.IMWRITE_PNG_COMPRESSION),0]) #save images in the folder
            cv2.imwrite(mask_path, m, [int(cv2.IMWRITE_PNG_COMPRESSION),0]) #save masks in the folder
                                                                            #with cv2.IMWRITE_PNG_COMPRESSION parameter the compression of the png image
                                                                            #can be controlled. The value 0 pruduces the lowest compression
            idx += 1

Creating the 'validation_images' and 'validation_masks' folders

In [ ]:
image_path_validation='/gdrive/MyDrive/validation_images/images/' # final images path
mask_path_validation='/gdrive/MyDrive/train/validation_masks/masks/' #final masks path

In [ ]:
H = 256 #height 
W = 256 #width
for idx, (x, y) in tqdm(enumerate(zip(val_x, val_y)), total=len(val_x)):
        """ Extracting the folder name and image name for each subject """
        dir_name = x.split("/")[-3]
        name = dir_name + "_" + x.split("/")[-1].split(".")[0] #extract the name of every subject's folder 
        """ Read the image and mask """
        x = cv2.imread(x, cv2.IMREAD_GRAYSCALE) #read the image
        y = cv2.imread(y, cv2.IMREAD_GRAYSCALE) #read the mask
        X = [x]
        Y = [y]
        idx = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H),interpolation = cv2.INTER_CUBIC)   #images are resized from 512x512 to 256x256 through bicubic interpolation
            m = cv2.resize(m, (W, H),interpolation = cv2.INTER_CUBIC)   #masks are resized from 512x512 to 256x256 through bicubic interpolation

            tmp_image_name = f"{name}.png"  #image and mask are saved with the same name 
            tmp_mask_name  = f"{name}.png" 

            image_path = os.path.join(image_path_validation, tmp_image_name)
            mask_path  = os.path.join(mask_path_validation, tmp_mask_name)
            print(tmp_image_name)

            cv2.imwrite(image_path,i, [int(cv2.IMWRITE_PNG_COMPRESSION),0]) #save images in the folder
            cv2.imwrite(mask_path, m, [int(cv2.IMWRITE_PNG_COMPRESSION),0]) #save masks in the folder
                                                                            #with cv2.IMWRITE_PNG_COMPRESSION parameter the compression of the png image
                                                                            #can be controlled. The value 0 pruduces the lowest compression
            idx += 1

##**Baseline U-Net**

###Data Augmentation

Connect to Drive

Importing Libraries


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
from matplotlib import pyplot as plt
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


Train and Validation paths

In [ ]:
train_img_path = "/gdrive/MyDrive/train_images" # training images path
train_mask_path = "/gdrive/MyDrive/train_masks" # training masks path

val_img_path = "/gdrive/MyDrive/validation_images" # validation images path
val_mask_path = "/gdrive/MyDrive/validation_masks" # validation masks path

Data Augmentation is applied simultaneously to trainining images and masks

In [ ]:
img_data_gen_args_train = dict(rescale=1./255,   #rescale the images
                     rotation_range=90,          #random rotation 
                     brightness_range=[0.3,1.5], # random change of brightness
                     width_shift_range=0.3,      # random width shifting
                     height_shift_range=0.3,     #random height shifting
                     shear_range=0.5,            #shear transformation
                     horizontal_flip=True,       #horizontal flip
                     vertical_flip=True,         # vertical flip
                     fill_mode='reflect')        # fill mode

mask_data_gen_args_train = dict(
                     rotation_range=90,          #random rotation 
                     brightness_range=[0.3,1.5], # random change of brightness
                     width_shift_range=0.3,      # random width shifting
                     height_shift_range=0.3,     #random height shifting
                     shear_range=0.5,            #shear transformation
                     horizontal_flip=True,       #horizontal flip
                     vertical_flip=True,         # vertical flip
                     fill_mode='reflect',       # fill mode

                     preprocessing_function = lambda x: np.where(x>0.5, 1, 0).astype(x.dtype) #Binarize the masks
                     
                     )  

image_data_generator_train = ImageDataGenerator(**img_data_gen_args_train)
mask_data_generator_train = ImageDataGenerator(**mask_data_gen_args_train)

Data Augmentation is not applied to validation set.
ImageDataGenerator is used to rescale the images and binarize the masks only.

In [ ]:
img_data_gen_args_val = dict(rescale=1./255) #rescale the images

mask_data_gen_args_val = dict(
                     preprocessing_function = lambda x: np.where(x>0.5, 1, 0).astype(x.dtype)
                     
                     ) #Binarize the masks. 

image_data_generator_val = ImageDataGenerator(**img_data_gen_args_val)
mask_data_generator_val = ImageDataGenerator(**mask_data_gen_args_val)

Defining the Generators.
The first generator is for training images and masks and the other one is for validation. These generators will be the input of the model.

In [ ]:
seed=42
batch_size=16
image_generator = image_data_generator_train.flow_from_directory(train_img_path, #path
                                                           seed=seed, # must be the same to ensure that images and masks are edited in the same way
                                                           batch_size=batch_size, #batch size
                                                           color_mode = 'grayscale', # grayscale. Channel number is equal to 1
                                                           target_size=(256,256), # height and the width
                                                           class_mode=None)  # Set this otherwise it returns multiple numpy arrays 
                                                                            

mask_generator = mask_data_generator_train.flow_from_directory(train_mask_path, 
                                                         seed=seed, 
                                                         batch_size=batch_size,
                                                         color_mode = 'grayscale',
                                                         target_size=(256,256)  , 
                                                         class_mode=None)


valid_img_generator = image_data_generator_val.flow_from_directory(val_img_path, 
                                                               seed=seed, 
                                                               batch_size=batch_size, 
                                                               color_mode = 'grayscale', 
                                                               target_size=(256,256),
                                                               class_mode=None) 


valid_mask_generator = mask_data_generator_val.flow_from_directory(val_mask_path, 
                                                               seed=seed, 
                                                               batch_size=batch_size, 
                                                               target_size=(256,256),
                                                               
                                                               color_mode = 'grayscale',  
                                                               class_mode=None)  


train_generator = zip(image_generator, mask_generator) # zip function to concatenate image and mask generators
val_generator = zip(valid_img_generator, valid_mask_generator)

Verifying generators

Show some example of augmented images and check if they are associated in correct way to the masks

In [ ]:

x, y = train_generator.__next__()

for i in range(0,8):
    image = x[i,:,:,0]
    mask= y[i,:,:,0]
    plt.subplot(1,2,1)
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(mask, cmap='gray')
    plt.axis('off')
    plt.show()

Check if images and masks are rescaled. 
Values must be between 0 and 1 for images and 0 or 1 for masks

In [ ]:
print(x.max())
print(y.max())

### Model

Importing Libraries

In [ ]:

from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate,UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K



Define the input shape of the model

In [ ]:
IMG_HEIGHT = x.shape[1]
IMG_WIDTH  = x.shape[2]
IMG_CHANNELS = x.shape[3]
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
print(input_shape)

Baseline U-Net Architecture


In [ ]:
inputs = tf.keras.layers.Input(input_shape)
#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1= tf.keras.layers.BatchNormalization()(c1)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
c1= tf.keras.layers.BatchNormalization()(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2= tf.keras.layers.BatchNormalization()(c2)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
c2= tf.keras.layers.BatchNormalization()(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3= tf.keras.layers.BatchNormalization()(c3)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
c3= tf.keras.layers.BatchNormalization()(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4= tf.keras.layers.BatchNormalization()(c4)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
c4= tf.keras.layers.BatchNormalization()(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5= tf.keras.layers.BatchNormalization()(c5)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
c5= tf.keras.layers.BatchNormalization()(c5)

#Expansive path 

u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

In [ ]:
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.summary()

Define the steps per epoch for training and validation required to train the model

In [ ]:
num_train_imgs = len(os.listdir("/gdrive/MyDrive/train_images/images")) #images path
num_val_images = len(os.listdir("/gdrive/MyDrive/val_images/images"))   #masks path

steps_per_epoch = num_train_imgs//batch_size
val_steps_per_epoch = num_val_images//batch_size


Define the model metrics and losses.
Dice Loss, IoU Loss, Tversky Loss, Focal Loss have been tested

In [ ]:
from keras import backend as K


def dice_coefficient(y_true, y_pred, smooth=0.0001): #smooth factor to avoid zero division
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)

    intersection = K.sum(y_true_f * y_pred_f)

    return ((2. * intersection + smooth) / (K.sum(y_true_f) +
            K.sum(y_pred_f) + smooth))


def dice_coefficient_loss(y_true, y_pred):
    return 1.0-dice_coefficient(y_true, y_pred)


def iou(y_true, y_pred):
    intersection = K.sum(K.abs(y_true * y_pred))
    sum_ = K.sum(K.square(y_true)) + K.sum(K.square(y_pred))
    jac = (intersection) / (sum_ - intersection)
    return jac

def iou_loss(y_true, y_pred):
    return 1-iou(y_true, y_pred)

def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7 #different values could be set
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)
def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)


Python libraries for focal loss

In [ ]:
!pip install focal-loss

In [ ]:
from focal_loss import BinaryFocalLoss

Define Learning Rate and Optimizer

In [ ]:
LR = 5e-5
optim = tf.keras.optimizers.Adam(LR) #Adaptive Moment Estimation Optimizer

Final Metrics used

In [ ]:
metrics = [iou, dice_coefficient, 'binary_accuracy','Recall','Precision']


Compile the model

Dice loss is chosen as loss function

In [ ]:
model.compile(optimizer=optim, loss=dice_coefficient_loss, metrics=metrics)


Add callback to save the best model.
The best model has the highest dice coefficient on the validation set

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/gdrive/MyDrive/chk/',
    save_weights_only=True,
    monitor='val_dice_coefficient',
    mode='max',
    save_best_only=True)

Train the model

In [ ]:
history=model.fit(train_generator,
          steps_per_epoch=steps_per_epoch,
          epochs=70,
          verbose=1,
          callbacks=model_checkpoint_callback,
          validation_data=val_generator,
          validation_steps=val_steps_per_epoch)

Save the model in .h5 format for future use

In [ ]:
model_path='/gdrive/MyDrive/model.h5'

In [ ]:
model.save(model_path)


### Evaluation on Validation Set

Importing Libraries



In [ ]:
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score
from tensorflow.keras.metrics import MeanIoU, Recall, Precision, BinaryAccuracy

     

Load previously saved model

In [ ]:
from keras.models import load_model
model = load_model(model_path,compile=False)

In [ ]:
a, b = val_generator.__next__() # Evaluation is performed directly in batches 


Prediction on random images of the validation set

In [ ]:
import random
test_img_number = random.randint(a.shape[0]) # random value from 0 to the number of images in the validation set
test_img = a[test_img_number] #testing image
ground_truth=b[test_img_number] # real mask
test_img_input=np.expand_dims(test_img, 0)
prediction = (model.predict(test_img_input)[0,:,:,0] > 0.5).astype(np.uint8) #prediction on testing image 


Show random examples


Testing image, predicted mask and real mask can be displayed

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(231)
plt.axis('off')
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')


plt.subplot(232)
plt.title('Real Mask ')
plt.axis('off')
plt.imshow(ground_truth[:,:,0], cmap='gray')
plt.subplot(233)
plt.axis('off')
plt.title('Predicted mask')
plt.imshow(prediction, cmap='gray')

plt.show()

Use the Mean Intersection Over Union metric from keras metrics

In [ ]:
from tensorflow.keras.metrics import MeanIoU


Compute the MeanIoU for a single image


In [ ]:
n_classes = 2 # 2 classes for binary segmentation
IOU_keras = MeanIoU(num_classes=n_classes)  

IOU_keras.update_state(ground_truth[:,:,0], prediction)
print("Mean IoU =", IOU_keras.result().numpy())

Compute the MeanIoU of each image in the validation set.
The MeanIoU for each image can be displayed and the final average value on all the predictions is computed

In [ ]:
import pandas as pd

IoU_values = []
for img in range(a.shape[0]): 
    temp_img = a[img]
    ground_truth=b[img]
    temp_img_input=np.expand_dims(temp_img, 0)
    prediction = (model.predict(temp_img_input)[0,:,:,0] > 0.5).astype(np.uint8)
    
    IoU = MeanIoU(num_classes=n_classes)
    IoU.update_state(ground_truth[:,:,0], prediction)
    IoU = IoU.result().numpy()
    IoU_values.append(IoU)

    print(IoU)
    
df = pd.DataFrame(IoU_values, columns=["IoU"])
df = df[df.IoU != 1.0]

mean_IoU = df.mean().values
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean IoU is: ", mean_IoU)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)

    

Binary Accuracy

In [ ]:

BinaryAccuracy_values = []
for img in range(a.shape[0]):
      temp_img = a[img]
      ground_truth=b[img]
      temp_img_input=np.expand_dims(temp_img, 0)
      prediction = (model.predict(temp_img_input)[0,:,:,0]> 0.5).astype(np.uint8)
      Accuracy=BinaryAccuracy()
      Accuracy.update_state(ground_truth[:,:,0], prediction)
      Accuracy = Accuracy.result().numpy()
      BinaryAccuracy_values.append(Accuracy)

      print(Accuracy)
      

df = pd.DataFrame(BinaryAccuracy_values, columns=["BinaryAccuracy"])
df = df[df.BinaryAccuracy != 1.0]    
mean_acc = df.mean().values
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range

print("Mean Binary Accuracy is: ", mean_acc)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)




Precision

In [ ]:

Precision_values = []
for img in range(a.shape[0]):

    temp_img = a[img]
    ground_truth=b[img]
    temp_img_input=np.expand_dims(temp_img, 0)
    prediction = (model.predict(temp_img_input)[0,:,:,0]> 0.5).astype(np.uint8)
    precision=precision_score(ground_truth[:,:,0], prediction, average='macro',zero_division=1) # to avoid zero division
    Precision_values.append (precision)
    print(precision)


df = pd.DataFrame(Precision_values, columns=["Precision"])
df = df[df.Precision != 1.0]    
mean_precision = df.mean().values
std=df.std()
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean Precision is: ", mean_precision)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)

Recall

In [ ]:

Recall_values = []
for img in range(a.shape[0]):

    temp_img = a[img]
    ground_truth=b[img]
    temp_img_input=np.expand_dims(temp_img, 0)
    prediction = (model.predict(temp_img_input)[0,:,:,0]> 0.5).astype(np.float32)
    recall=recall_score(ground_truth[0,:,:,0], prediction, average='macro',zero_division=1) # to avoid zero division
    Recall_values.append (recall)
    print(recall)


df = pd.DataFrame(Recall_values, columns=["Recall"])
df = df[df.Recall != 1.0]    
boxplot = df.boxplot(grid=False,vert=True,color='blue')
mean_rec = df.mean().values
std=df.std()
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean Recall is: ", mean_rec)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)

### Evaluation on Test Set

21 subjects are used to test the model performance. Images and masks of testing dataset were preprocessed in the same way of training and validation ones.
21 folders were created each containing the images and the corresponding masks. 

Connect to Drive

In [ ]:
from google.colab import drive

drive.mount("/gdrive")


Importing Libraries

In [ ]:
from keras.models import load_model
import os
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score
from tensorflow.keras.metrics import MeanIoU, Recall, Precision, BinaryAccuracy, IoU


Load previously saved model


In [ ]:
model_path='/gdrive/MyDrive/model.h5'

In [ ]:
model = load_model(model_path, compile=False)

Test Images and Masks paths

In [ ]:
test_img_path = "/test_images/images" #  images path
test_mask_path = "/test_masks/masks"  #  masks path

ImageDataGenerator is used to rescale the images and binarize the masks.

In [ ]:
img_data_gen_args_test = dict(rescale=1.0 / 255)

mask_data_gen_args_test = dict(
    preprocessing_function=lambda x: np.where(x > 0.5, 1, 0).astype(x.dtype)
)  # Binarize the output again.

image_data_generator_test = ImageDataGenerator(**img_data_gen_args_test)
mask_data_generator_test = ImageDataGenerator(**mask_data_gen_args_test)

Ordering in correct way the images and the masks

In [ ]:
def sort_img_names(dir: str):
    names = [os.path.join(dir, x) for x in os.listdir(dir)]
    names = sorted(names, key=lambda x: int(x.split("/")[-1].split(".")[0]))
    return names


test_img_names = pd.DataFrame(sort_img_names(test_img_path), columns=["filename"])
test_mask_names = pd.DataFrame(sort_img_names(test_mask_path), columns=["filename"])

In [ ]:
n_of_images= len(test_img_names)
print(n_of_images)

Batch size is equal to the number of images (and masks) in test set.

In [ ]:
batch_size= n_of_images 

In [ ]:
seed = 42
test_img_generator = image_data_generator_test.flow_from_dataframe(
    test_img_names,
    seed=seed,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=False, # to mantain the correct order of images and masks
    target_size=(256, 256),
    class_mode=None,
) 
test_mask_generator = mask_data_generator_test.flow_from_dataframe(
    test_mask_names,
    seed=seed,
    batch_size=batch_size,
    target_size=(256, 256),
    shuffle=False, #to mantain the correct order of images and masks
    color_mode="grayscale",  
    class_mode=None,
) 


test_generator = zip(valid_img_generator, valid_mask_generator)

In [ ]:
a, b = test_generator.__next__()


Prediction on random images of the test set

In [ ]:

  test_img_number = random.randint(a.shape[0]) #random number
  test_img = a[test_img_number] #testing image
  ground_truth=b[test_img_number] #real mask
  test_img_input=np.expand_dims(test_img, 0) 
  prediction = (model.predict(test_img_input)[0,:,:,0] > 0.5 ).astype(np.uint8) #predicted mask
  plt.figure(figsize=(16, 8))
  plt.subplot(231)
  plt.axis('off')
  plt.title('Testing Image')
  plt.imshow(test_img[:,:,0], cmap='gray')



  plt.subplot(232)
  plt.title('Real Mask ')
  plt.axis('off')
  plt.imshow(ground_truth[:,:,0], cmap='gray')
  plt.subplot(233)

  plt.axis('off')

  plt.title('Predicted mask')
  plt.imshow(prediction, cmap='gray')

  plt.show()

MeanIoU for a single image

In [ ]:
n_classes = 2
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(ground_truth[:,:,0], prediction)
print("Mean IoU =", IOU_keras.result().numpy())

MeanIoU for all images

In [ ]:
import pandas as pd

IoU_values = []
for img in range(0, a.shape[0]):
    temp_img = a[img]
    ground_truth=b[img]
    temp_img_input=np.expand_dims(temp_img, 0)
    prediction = (model.predict(temp_img_input)[0,:,:,0] > 0.5).astype(np.uint8)
    
    IoU = MeanIoU(num_classes=n_classes)
    IoU.update_state(ground_truth[:,:,0], prediction)
    IoU = IoU.result().numpy()
    IoU_values.append(IoU)

    print(IoU)
    
df = pd.DataFrame(IoU_values, columns=["IoU"])
df = df[df.IoU != 1.0]

mean_IoU = df.mean().values
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean IoU is: ", mean_IoU)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)


Precision for all images

In [ ]:

Precision_values = []
for img in range(a.shape[0]):

    temp_img = a[img]
    ground_truth=b[img]
    temp_img_input=np.expand_dims(temp_img, 0)
    prediction = (model.predict(temp_img_input)[0,:,:,0]> 0.5).astype(np.uint8)
    precision=precision_score(ground_truth[:,:,0], prediction, average='macro',zero_division=1) # to avoid zero division
    Precision_values.append (precision)
    print(precision)


df = pd.DataFrame(Precision_values, columns=["Precision"])
df = df[df.Precision != 1.0]    
mean_precision = df.mean().values
std=df.std()
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean Precision is: ", mean_precision)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)

Recall for all images

In [ ]:


Recall_values = []
for img in range(a.shape[0]):

    temp_img = a[img]
    ground_truth=b[img]
    temp_img_input=np.expand_dims(temp_img, 0)
    prediction = (model.predict(temp_img_input)[0,:,:,0]> 0.5).astype(np.float32)
    recall=recall_score(ground_truth[0,:,:,0], prediction, average='macro',zero_division=1) # to avoid zero division
    Recall_values.append (recall)
    print(recall)


df = pd.DataFrame(Recall_values, columns=["Recall"])
df = df[df.Recall != 1.0]    
boxplot = df.boxplot(grid=False,vert=True,color='blue')
mean_rec = df.mean().values
std=df.std()
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean Recall is: ", mean_rec)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)

Binary Accuracy for all images

In [ ]:

BinaryAccuracy_values = []
for img in range(a.shape[0]):
      temp_img = a[img]
      ground_truth=b[img]
      temp_img_input=np.expand_dims(temp_img, 0)
      prediction = (model.predict(temp_img_input)[0,:,:,0]> 0.5).astype(np.uint8)
      Accuracy=BinaryAccuracy()
      Accuracy.update_state(ground_truth[:,:,0], prediction)
      Accuracy = Accuracy.result().numpy()
      BinaryAccuracy_values.append(Accuracy)

      print(Accuracy)
      

df = pd.DataFrame(BinaryAccuracy_values, columns=["BinaryAccuracy"])
df = df[df.BinaryAccuracy != 1.0]    
mean_acc = df.mean().values
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range

print("Mean Binary Accuracy is: ", mean_acc)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)


###Final Metrics excluding black masks and first slices



In [ ]:

IoU_values = []
n_pixel_true=[]
n_pixel_predicted=[]
for img in range(70,a.shape[0]): #70 represents the number of slice in which the IoU starts to be high. Here the predicted masks and the real masks are similar. This starting value changes from patient to patient.
    temp_img = a[img]
    ground_truth=b[img]
    if (ground_truth.max()>0):
        temp_img_input=np.expand_dims(temp_img, 0)
        prediction = (model.predict(temp_img_input)[0,:,:,0]>0.5).astype(np.uint8) # if there are values greater than 0 , the real mask contains useful information and cannot be discarded
      
        n_pixel_true1=ground_truth.sum() #real area
        n_pixel_predicted1=prediction.sum() #predicted area
        IoU = MeanIoU(num_classes=n_classes)
        IoU.update_state(ground_truth[:,:,0], prediction)
        IoU = IoU.result().numpy()
        IoU_values.append(IoU)
        n_pixel_true.append(n_pixel_true1)
        n_pixel_predicted.append(n_pixel_predicted1)



        print(IoU)
  

df = pd.DataFrame(IoU_values, columns=["IoU"])
df = df[df.IoU != 1.0]    
mean_IoU = df.mean().values
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print('Number of masks is', len(IoU_values))
print("Mean IoU is: ", mean_IoU)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)
     

Binary Accuracy


In [ ]:

BinaryAccuracy_values = []
for img in range(70,a.shape[0]): #70 represents the number of slice in which the IoU starts to be high. Here the predicted masks and the real masks are similar. This starting value changes from patient to patient.
    temp_img = a[img]
    ground_truth=b[img]
    if (ground_truth.max()>0):
        temp_img_input=np.expand_dims(temp_img, 0)
        prediction = (model.predict(temp_img_input)[0,:,:,0]>0.5).astype(np.uint8) # if there are values greater than 0 , the real mask contains useful information and cannot be discarded


        Accuracy=BinaryAccuracy()
        Accuracy.update_state(ground_truth[:,:,0], prediction)
        Accuracy = Accuracy.result().numpy()
        BinaryAccuracy_values.append(Accuracy)

        print(Accuracy)
      

df = pd.DataFrame(BinaryAccuracy_values, columns=["BinaryAccuracy"])
df = df[df.BinaryAccuracy != 1.0]    
mean_acc = df.mean().values
std=df.std()
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean Accuracy is: ", mean_acc)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)

Precision

In [ ]:


Precision_values = []
for img in range(70,a.shape[0]): #70 represents the number of slice in which the IoU starts to be high. Here the predicted masks and the real masks are similar. This starting value changes from patient to patient.
    temp_img = a[img]
    ground_truth=b[img]
    if (ground_truth.max()>0):
        temp_img_input=np.expand_dims(temp_img, 0)
        prediction = (model.predict(temp_img_input)[0,:,:,0]>0.5).astype(np.uint8) # if there are values greater than 0 , the real mask contains useful information and cannot be discarded


        precision=precision_score(ground_truth[:,:,0], prediction, average='macro',zero_division=1)
        Precision_values.append (precision)
        print(precision)


df = pd.DataFrame(Precision_values, columns=["Precision"])
df = df[df.Precision != 1.0]    
mean_precision = df.mean().values
std=df.std()
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean Precision is: ", mean_precision)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)
     

Recall

In [ ]:


Recall_values = []
for img in range(70,a.shape[0]): #70 represents the number of slice in which the IoU starts to be high. Here the predicted masks and the real masks are similar. This starting value changes from patient to patient.
    temp_img = a[img]
    ground_truth=b[img]
    if (ground_truth.max()>0):
        temp_img_input=np.expand_dims(temp_img, 0)
        prediction = (model.predict(temp_img_input)[0,:,:,0]>0.5).astype(np.uint8) # if there are values greater than 0 , the real mask contains useful information and cannot be discarded

        recall=recall_score(ground_truth[:,:,0], prediction, average='macro',zero_division=1)
        Recall_values.append (recall)
        print(recall)


df = pd.DataFrame(Recall_values, columns=["Recall"])
df = df[df.Recall != 1.0]    
mean_rec = df.mean().values
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean Recall is: ", mean_rec)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)

## **Sequential U-Net**

Connect to Drive

In [ ]:
from google.colab import drive

drive.mount("/gdrive")

Importing Libraries

In [ ]:
import os
from IPython.core.completer import time
from natsort import natsorted
from tensorflow.keras.preprocessing.image import (
    array_to_img,
    img_to_array,
    load_img,
)
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np

from matplotlib import pyplot as plt
import tensorflow as tf

     

###Data Loader and Data Augmentation

Through the current Data Loader it is possible to take the time series Im(t), Im(t + 1),Im(t + 2), and the mask associated with the last of this sequence, with overlap.
Data Augmentation is performed.

In [ ]:


class DataLoader:
    def __init__(
        self,
        *,
        data_path,
        timesteps: int = 3, #time steps
        batch_size: int = 12, #batch size
        squeeze=False, 
        augmentation=True,
        sort_method="natsort" #sort method
    ):

        self.do_augmentation = augmentation
        self.timesteps = timesteps
        self.batch_size = 12
        self.squeeze = squeeze
        self.data_path = data_path
        train_img_file_names = [
            os.path.join(data_path, fn)
            for fn in os.listdir(
                data_path
            )  #
        ]
        def sort_by_number(file_names:str):
            return sorted(names, key=lambda x: int(x.split('/')[-1].split('.')[0]))

        train_img_file_names = natsorted(train_img_file_names) if sort_method == "natsort" else sort_by_number(train_img_file_names) # sorting images
     
        train_mask_file_names = [
            fn.replace("images", "masks") for fn in train_img_file_names   #sorting masks
        ]
        self.imgs = (
            np.array(
                [
                    img_to_array(load_img(fn, color_mode="grayscale"))
                    for fn in train_img_file_names
                ]
            )
            / 255 #rescale the images
        )
        self.masks = (
            np.array(
                [
                    img_to_array(load_img(fn, color_mode="grayscale"))
                    for fn in train_mask_file_names
                ]
            )
            / 255 #rescale the masks
            > 0.5 #Binarize the masks
        ).astype(float)
        #Data Augmentation
        self.augmentation = tf.keras.Sequential(   
            [
                layers.RandomFlip("horizontal_and_vertical"), #random flip in horizontal and vertical axis
                layers.RandomRotation(0.2), #random rotation
            ]
        )

    def __iter__(self):
        return self
    #Creating the Generators
    def __next__(self):
        xs = []
        ys = []
        for _ in range(self.batch_size):
            index = np.random.randint(0, len(self.imgs) - self.timesteps)
            xs.append(self.imgs[index : index + self.timesteps])
            ys.append(self.masks[index + self.timesteps])
        x, y = np.array(xs), np.array(ys)
        x = x.squeeze(-1).transpose(0, 2, 3, 1)
        xy = tf.concat((x, y), axis=-1)
        if self.do_augmentation:
            xy = self.augmentation(xy)

        x = xy[:, :, :, :3]
        y = xy[:, :, :, 3:]
        if not self.squeeze:
            x = tf.transpose(x, perm=(0, 3, 1, 2))[..., None]
            y = tf.expand_dims(y, 1)
        return x, y
     

Defining the Generators. The first generator is for training images and masks and the other one is for validation. These generators will be the input of the model.

In [ ]:
train_loader = DataLoader(
    data_path=os.path.join(
        "/gdrive/MyDrive/" , "train_images", "images" # training images path
    ),
    batch_size=12,
    squeeze=False, #if squeeze is True, the output tensors are 2D , if False the output tensors are 3D
    augmentation=True, #Data Augmentation is applied simultaneously to trainining images and masks
)

val_loader = DataLoader(
    data_path=os.path.join(
        "/gdrive/MyDrive/",
        "validation_images", "images" # validation images path
    ),
    batch_size=12,
    squeeze=False,
    augmentation=False, #Data Augmentation is not applied to validation set. Data Loader is used to produce the final tensor with the correct shape and to rescale the images and binarize the masks.


)

Verifying generators

Show some example of augmented images and check if they are associated in correct way to the masks



In [ ]:

x, y = train_loader.__next__()

for i in range(0,12):
    image = x[i,0,:,:,0]
    mask= y[i,0,:,:,0]
    plt.subplot(1,2,1)
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(mask, cmap='gray')
    plt.axis('off')
    plt.show()
     

Check if images and masks are rescaled. Values must be between 0 and 1 for images and 0 or 1 for masks



In [ ]:

print(x.max())
print(y.max())

Check if tensors have the correct shape.
Images shape must be equal to ( *batch size*, *time steps*, *height*, *width*, *number of channels* )

Masks shape must be equal to ( *batch size*, *time steps*,*height*,*width*, *number of channels*)

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:

x, y = val_loader.__next__()

for i in range(0,3):
    image = x[i,0,:,:,0]
    mask= y[i,0,:,:,0]
    plt.subplot(1,2,1)
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(mask, cmap='gray')
    plt.axis('off')
    plt.show()

###Model

Importing Libraries


In [ ]:
from tensorflow.keras.layers import Conv2D, TimeDistributed,Dropout,Input, Dense,\
    BatchNormalization, GRU, Layer, Flatten,MaxPooling2D, concatenate,Lambda
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from keras import layers
from keras import models
from tensorflow.python.keras.layers import ConvLSTM2D
from keras import backend as K


     

Define the input shape of the model



In [ ]:
IMG_TIME_STEPS= x.shape[1]
IMG_HEIGHT = x.shape[2]
IMG_WIDTH  = x.shape[3]
IMG_CHANNELS = x.shape[4]
input_shape = (IMG_TIME_STEPS,IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
print(input_shape)

In [ ]:

input_l = layers.Input(shape=(input_shape))
#Contraction path
x =  (layers.TimeDistributed(layers.Conv2D( 16, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal', activation='relu'))) (input_l)
x=layers.TimeDistributed(layers.BatchNormalization())(x)
conv1 = layers.TimeDistributed( layers.Conv2D( 16, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal', activation='relu' ) ) (x)
conv1=layers.TimeDistributed(layers.BatchNormalization())(conv1)
x=layers.TimeDistributed(layers.MaxPooling2D(pool_size=(2,2)))(conv1)
x = layers.TimeDistributed( layers.Conv2D( 32, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal',activation='relu' ) ) (x)
x=layers.TimeDistributed(layers.BatchNormalization())(x)
conv2 = layers.TimeDistributed( layers.Conv2D( 32, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal', activation='relu' ) ) (x)
conv2=layers.TimeDistributed(layers.BatchNormalization())(conv2)
x=layers.TimeDistributed(layers.MaxPooling2D(pool_size=(2,2)))(conv2)
x = layers.TimeDistributed( layers.Conv2D( 64, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal', activation='relu' ) ) (x)
x=layers.TimeDistributed(layers.BatchNormalization())(x)
conv3 = layers.TimeDistributed( layers.Conv2D( 64, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal', activation='relu' ) ) (x)
conv3=layers.TimeDistributed(layers.BatchNormalization())(conv3)
x=layers.TimeDistributed(layers.MaxPooling2D(pool_size=(2,2)))(conv3)
x = layers.TimeDistributed( layers.Conv2D( 128, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal', activation='relu' ) ) (x)
x=layers.TimeDistributed(layers.BatchNormalization())(x)
conv4 = layers.TimeDistributed( layers.Conv2D( 128, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal', activation='relu')) (x)
conv4=layers.TimeDistributed(layers.BatchNormalization())(conv4)
x=layers.TimeDistributed(layers.MaxPooling2D(pool_size=(2,2)))(conv4)
x = layers.TimeDistributed( layers.Conv2D( 256, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal', activation='relu' ) ) (x)
x=layers.TimeDistributed(layers.BatchNormalization())(x)
conv5 = layers.TimeDistributed( layers.Conv2D( 256, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal',activation='relu' ) ) (x)
conv5=layers.TimeDistributed(layers.BatchNormalization())(conv5)
# LSTM component
x=layers.ConvLSTM2D(256,kernel_size=(3,3),padding='same',strides=(1,1),return_sequences=True,recurrent_dropout=0.2))(conv5)
#Expansive path
up1 = layers.TimeDistributed( layers.Conv2DTranspose(128,kernel_size=(3,3),padding='same',strides=(2,2)))(x)
concat1 = layers.concatenate([up1, conv4])
x = layers.TimeDistributed( layers.Conv2D( 128, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal', activation='relu' ) ) (concat1)
x=layers.TimeDistributed(layers.BatchNormalization())(x)
x = layers.TimeDistributed( layers.Conv2D( 128, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal', activation='relu') ) (x)
x=layers.TimeDistributed(layers.BatchNormalization())(x)
up2 = layers.TimeDistributed( layers.Conv2DTranspose( 64,kernel_size=(3,3),padding='same',strides=(2,2)))(x)
concat2 = layers.concatenate([up2, conv3])
x = layers.TimeDistributed( layers.Conv2D( 64, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal', activation='relu' ) ) (concat2)
x=layers.TimeDistributed(layers.BatchNormalization())(x)
x = layers.TimeDistributed( layers.Conv2D( 64, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal', activation='relu' ) ) (x)
x=layers.TimeDistributed(layers.BatchNormalization())(x)
up3 = layers.TimeDistributed( layers.Conv2DTranspose( 32,kernel_size=(3,3),padding='same',strides=(2,2)))(x)
concat3 = layers.concatenate([up3, conv2])
x = layers.TimeDistributed( layers.Conv2D( 32, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal',activation='relu') ) (concat3)
x=layers.TimeDistributed(layers.BatchNormalization())(x)
x = layers.TimeDistributed( layers.Conv2D( 32, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal',activation='relu') ) (x)
x=layers.TimeDistributed(layers.BatchNormalization())(x)
up4= layers.TimeDistributed( layers.Conv2DTranspose( 16,kernel_size=(3,3),padding='same',strides=(2,2)))(x)
concat4 = layers.concatenate([up4, conv1])
x = layers.TimeDistributed( layers.Conv2D( 16, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal', activation='relu') ) (concat4)
x=layers.TimeDistributed(layers.BatchNormalization())(x)
x = layers.TimeDistributed( layers.Conv2D( 16, kernel_size=(3, 3),padding='same',strides=(1,1),kernel_initializer='he_normal',activation='relu') ) (x)
x=layers.TimeDistributed(layers.BatchNormalization())(x)
#LSTM component
x=layers.ConvLSTM2D(16,kernel_size=(3,3),padding='same',strides=(1,1),return_sequences=False,recurrent_dropout=0.2))(x)
x=tf.expand_dims(x,axis=1)
out = layers.Conv2D( 1, kernel_size=(1, 1),padding='same',strides=(1,1), activation='sigmoid' )  (x)


In [ ]:

model = models.Model(inputs=input_l, outputs=out)
model.summary()
     

Define the steps per epoch for training and validation required to train the model



In [ ]:


num_train_imgs = len(os.listdir("/gdrive/MyDrive/train_images/images") #images path
num_val_images = len(os.listdir("/gdrive/MyDrive/val_images/images") #masks path
                     
steps_per_epoch = num_train_imgs//batch_size
val_steps_per_epoch = num_val_images//batch_size


Define the model metrics and losses. Dice Loss, IoU Loss, Tversky Loss, Focal Loss have been tested



In [ ]:

def dice_coefficient(y_true, y_pred, smooth=0.0001): #smooth factor to avoid zero division
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)

    intersection = K.sum(y_true_f * y_pred_f)

    return ((2. * intersection + smooth) / (K.sum(y_true_f) +
            K.sum(y_pred_f) + smooth))


def dice_coefficient_loss(y_true, y_pred):
    return 1.0-dice_coefficient(y_true, y_pred)


def iou(y_true, y_pred):
    intersection = K.sum(K.abs(y_true * y_pred))
    sum_ = K.sum(K.square(y_true)) + K.sum(K.square(y_pred))
    jac = (intersection) / (sum_ - intersection)
    return jac

def iou_loss(y_true, y_pred):
    return 1-iou(y_true, y_pred)

def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)
def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)

Python libraries for focal loss



In [ ]:
!pip install focal-loss


In [ ]:

from focal_loss import BinaryFocalLoss

Define Learning Rate and Optimizer



In [ ]:
LR = 5e-5
optim = tf.keras.optimizers.Adam(LR) #Adaptive Moment Estimation Optimizer

Final Metrics used



In [ ]:
metrics = [iou, dice_coefficient, 'binary_accuracy','Recall','Precision']

Compile the model




Dice loss is chosen as loss function


In [ ]:

model.compile(optimizer=optim, loss=dice_coefficient_loss, metrics=metrics)

     

Add callback to save the best model. The best model has the highest dice coefficient on the validation set



In [ ]:

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/gdrive/MyDrive/chk/',
    save_weights_only=True,
    monitor='val_dice_coefficient',
    mode='max',
    save_best_only=True)

Train the model



In [ ]:

history=model.fit(train_loader,
          steps_per_epoch=steps_per_epoch,
          epochs=40,
          verbose=1,
          validation_data=val_loader,
          callbacks=model_checkpoint_callback,
          validation_steps=val_steps_per_epoch)

Save the model in .h5 format for future use



In [ ]:

model_path='/gdrive/MyDrive/model.h5'

In [ ]:
model.save(model_path)

###Evaluation on Validation Set

Load previously saved model



In [ ]:
from keras.models import load_model
model = load_model(model_path,compile=False)

Evaluation is performed directly in batches 

In [ ]:

class DataLoader:
    def __init__(
        self,
        *,
        data_path,
        timesteps: int = 3,
        batch_size: int = 477, 
        squeeze=False,
        augmentation=True,
        sort_method="natsort" 
    ):

        self.do_augmentation = augmentation
        self.timesteps = timesteps
        self.batch_size = 477 # number of images and masks within the validation set
        self.squeeze = squeeze
        self.data_path = data_path
        train_img_file_names = [
            os.path.join(data_path, fn)
            for fn in os.listdir(
                data_path
            )  
        ]
        def sort_by_number(file_names:str):
            return sorted(names, key=lambda x: int(x.split('/')[-1].split('.')[0]))

        train_img_file_names = natsorted(train_img_file_names) if sort_method == "natsort" else sort_by_number(train_img_file_names)

        train_mask_file_names = [
            fn.replace("images", "masks") for fn in train_img_file_names
        ]
        self.imgs = (
            np.array(
                [
                    img_to_array(load_img(fn, color_mode="grayscale")) #grayscale. Channel number is equal to 1
                    for fn in train_img_file_names
                ]
            )
            / 255 #rescale the images
        )
        self.masks = (
            np.array(
                [
                    img_to_array(load_img(fn, color_mode="grayscale")) #grayscale. Channel number is equal to 1
                    for fn in train_mask_file_names
                ]
            )
            / 255 #recsale the masks
            > 0.5 #binarize the masks
        ).astype(float)
        self.augmentation = tf.keras.Sequential(
            [
                layers.RandomFlip("horizontal_and_vertical"),
                layers.RandomRotation(0.2),
            ]
        )

    def __iter__(self):
        return self
   #Creating the Generator

    def __next__(self):
        xs = []
        ys = []
        for _ in range(self.batch_size):
            index = np.random.randint(0, len(self.imgs) - self.timesteps)
            xs.append(self.imgs[index : index + self.timesteps])
            ys.append(self.masks[index + self.timesteps])
        x, y = np.array(xs), np.array(ys)
        x = x.squeeze(-1).transpose(0, 2, 3, 1)
        xy = tf.concat((x, y), axis=-1)
        if self.do_augmentation:
            xy = self.augmentation(xy)

        x = xy[:, :, :, :3]
        y = xy[:, :, :, 3:]
        if not self.squeeze:
            x = tf.transpose(x, perm=(0, 3, 1, 2))[..., None]
            y = tf.expand_dims(y, 1)
        return x, y
     

In [ ]:

val_loader = DataLoader(
    data_path=os.path.join(
        "/gdrive/MyDrive/",
        "validation_images", "images"
    ),
    batch_size=477, # number of images and masks within the validation set
    squeeze=False,
    augmentation=False,
)

In [ ]:
a, b = val_loader.__next__()

Prediction on random images in the validation set



In [ ]:

import random

test_img_number = random.randint(a.shape[0]) # random value from 0 to the number of images in the validation set
test_img = a[test_img_number] #testing image
ground_truth=b[test_img_number] # real mask
test_img_input=np.expand_dims(test_img, 0) #prediction on testing image 
prediction = (model.predict(test_img_input)[0,0,:,:,0] > 0.5).astype(np.uint8)
     

Show random examples



Testing image, predicted mask and real mask can be displayed



In [ ]:
  plt.figure(figsize=(16, 8))
  plt.subplot(231)
  plt.axis('off')
  plt.title('Testing Image')
  plt.imshow(test_img[0,:,:,0], cmap='gray')


  plt.subplot(232)
  plt.title('Real Mask ')
  plt.axis('off')
  plt.imshow(ground_truth[0,:,:,0], cmap='gray')
  plt.subplot(233)
  plt.axis('off')
  plt.title('Predicted mask')
  plt.imshow(prediction, cmap='gray')

  plt.show()

Use the Mean Intersection Over Union metric from keras metrics



In [ ]:
from tensorflow.keras.metrics import MeanIoU


Compute the MeanIoU for a single image



In [ ]:

n_classes = 2 # 2 classes for binary segmentation
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(ground_truth[:,:,0], prediction)
print("Mean IoU =", IOU_keras.result().numpy())
     

Compute the meanIoU of each image in the validation set. The MeanIoU for each image can be displayed and the final average value on all the predictions is computed



In [ ]:

import pandas as pd

IoU_values = []
for img in range(0, a.shape[0]):
    temp_img = a[img]
    ground_truth=b[img]
    temp_img_input=np.expand_dims(temp_img, 0)
    prediction = (model.predict(temp_img_input)[0,0,:,:,0]> 0.5).astype(np.uint8)
    
    IoU = MeanIoU(num_classes=n_classes)
    IoU.update_state(ground_truth[0,:,:,0], prediction)
    IoU = IoU.result().numpy()
    IoU_values.append(IoU)

    print(IoU)
    


df = pd.DataFrame(IoU_values, columns=["IoU"])
df = df[df.IoU != 1.0]    
mean_IoU = df.mean().values
print("Mean IoU is: ", mean_IoU)

###Evaluation on Test Set

21 subjects are used to test the model performance. Images and masks of testing dataset were preprocessed in the same way of training and validation ones. 21 folders were created each containing the images and the corresponding masks.

Connect to Drive



In [ ]:

from google.colab import drive

drive.mount("/gdrive")

Importing Libraries



In [ ]:

from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score
from tensorflow.keras.metrics import MeanIoU, Recall, Precision, BinaryAccuracy, IoU
from matplotlib import pyplot as plt
import random
import pandas as pd

import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm_notebook, tnrange
from glob import glob
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
import os
from IPython.core.completer import time
from skimage.transform import resize
from natsort import natsorted
from tensorflow.keras.preprocessing.image import (
    array_to_img,
    img_to_array,
    load_img,
)
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
import pickle
from tqdm import tqdm

from tensorflow.keras import backend as K


In [ ]:
class DataLoader:
    def __init__(
        self,
        *,
        data_path,
        paziente: str,
        timesteps: int = 3,
        squeeze=False,
        augmentation=False,
        shuffle=False,
        sort_method="natsort", 
    ):
        self.shuffle = shuffle
        self.do_augmentation = augmentation
        self.timesteps = timesteps
        self.batch_size = batch_size
        self.squeeze = squeeze
        self.data_path = data_path
        train_img_file_names = [
            os.path.join(data_path, fn)
            for fn in os.listdir(
                data_path
            ) 
        ]

        train_img_file_names = (
            natsorted(train_img_file_names)
            if sort_method == "natsort"
            else sort_by_number(train_img_file_names)
        )
        self.filenames = train_img_file_names
        train_mask_file_names = [
            fn.replace("images", "masks") for fn in train_img_file_names
        ]
        if not os.path.exists(f"{paziente}imgs.pickle"):   #save sorted images into drive
            self.imgs = (
                np.array(
                    [
                        resize(
                            img_to_array(load_img(fn, color_mode="grayscale")),
                            (resize_to, resize_to),
                        )
                        for fn in train_img_file_names
                    ]
                )
                / 255 # rescale the images 
            )
            with open(f"{paziente}imgs.pickle", "wb") as f: #load sorted images from drive
                pickle.dump(self.imgs, f)
        else:
            print("Loading imgs from pickle")
            with open(f"{paziente}imgs.pickle", "rb") as f:
                self.imgs = pickle.load(f)
        if not os.path.exists(f"{paziente}_masks.pickle"): #save sorted masks into drive
            self.masks = (
                np.array(
                    [
                        resize(
                            img_to_array(load_img(fn, color_mode="grayscale")),
                            (resize_to, resize_to),
                        )
                        for fn in train_mask_file_names
                    ]
                )
                / np.array(255) #rescale the masks
                > 0.5 #Binarize the masks
            ).astype(float)
            with open(f"{paziente}masks.pickle", "wb") as f:# load sorted masks from drive
                pickle.dump(self.masks, f)
        else:
            print("Loading masks from pickle file")
            with open(f"{paziente}masks.pickle", "rb") as f:
                self.masks = pickle.load(f)

        self.augmentation = tf.keras.Sequential(
            [
                layers.RandomFlip("horizontal_and_vertical"),
                layers.RandomRotation(0.2),
            ]
        )
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        imgs = np.stack(
            [
                self.imgs[i : i + 3]
                for i in range(0, len(self.imgs))
                if i + 3 < len(self.imgs)
            ]
        )
        masks = np.stack(
            [
                self.masks[i, None]
                for i in range(0, len(self.masks))
                if i + 3 < len(self.masks)
            ]
        )
        return imgs, masks


In [ ]:

test_loader = DataLoader(
        data_path=os.path.join(
                   "/gdrive/MyDrive/",
        "test_images", "images",
        ),

        paziente="1", 
        squeeze=False,
        augmentation=False,
        shuffle=False,
        sort_method="number",

)

In [ ]:
a, b = test_loader.__next__()

Load previously saved model


In [ ]:

model_path='/gdrive/MyDrive/model.h5'

In [ ]:

model = load_model(model_path, compile=False)


Prediction on random images of the test set



In [ ]:


  test_img_number = random.randint(0,a.shape[0]) #random number
  test_img = a[test_img_number] #Testing Image
  ground_truth = b[test_img_number] #Real Mask
  test_img_input = np.expand_dims(test_img, 0)

  prediction = (model.predict(test_img_input)[0, 0, :, :, 0] > 0.5).astype(np.uint8) #Predicted mask
  plt.figure(figsize=(16, 8))
  
  plt.subplot(231)
  plt.axis("off")
  plt.title("Testing Image")
  plt.imshow(test_img[0, :, :, 0], cmap="gray")

  plt.subplot(232)
  plt.title("Real Mask ")
  plt.axis("off")
  plt.imshow(ground_truth[0, :, :, 0], cmap="gray")
  plt.subplot(233)
  plt.axis("off")

  plt.title("Predicted mask")
  plt.imshow(prediction, cmap="gray")

  plt.show()

MeanIoU for a single image



In [ ]:
from tensorflow.keras.metrics import MeanIoU
n_classes = 2
IOU_keras = MeanIoU(num_classes=n_classes)
IOU_keras.update_state(ground_truth[0, :, :, 0], prediction)
print("Mean IoU =", IOU_keras.result().numpy())

MeanIoU for all images




In [ ]:

import pandas as pd

IoU_values = []
for img in range(a.shape[0]):
    temp_img = a[img]
    ground_truth = b[img]
    temp_img_input = np.expand_dims(temp_img, 0)
    prediction = (model.predict(temp_img_input)[0, 0, :, :, 0] > 0.5).astype(np.uint8)

    IoU = MeanIoU(num_classes=n_classes)
    IoU.update_state(ground_truth[0, :, :, 0], prediction)
    IoU = IoU.result().numpy()
    IoU_values.append(IoU)

    print(IoU)


df = pd.DataFrame(IoU_values, columns=["IoU"])
df = df[df.IoU != 1.0]

mean_IoU = df.mean().values
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean IoU is: ", mean_IoU)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)

Binary Accuracy for all images

In [ ]:
import pandas as pd
BinaryAccuracy_values = []
for img in range(a.shape[0]):
      temp_img = a[img]
      ground_truth=b[img]
      temp_img_input=np.expand_dims(temp_img, 0)
      prediction = (model.predict(temp_img_input)[0,0,:,:,0]> 0.5).astype(np.float32)
      Accuracy=BinaryAccuracy()
      Accuracy.update_state(ground_truth[:,:,0], prediction)
      Accuracy = Accuracy.result().numpy()
      BinaryAccuracy_values.append(Accuracy)

      print(Accuracy)
      

df = pd.DataFrame(BinaryAccuracy_values, columns=["BinaryAccuracy"])
df = df[df.BinaryAccuracy != 1.0]    
mean_acc = df.mean().values
std=df.std()
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean Accuracy is: ", mean_acc)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)



Recall for all images

In [ ]:

Recall_values = []
for img in range(a.shape[0]):

    temp_img = a[img]
    ground_truth=b[img]
    temp_img_input=np.expand_dims(temp_img, 0)
    prediction = (model.predict(temp_img_input)[0,0,:,:,0]> 0.5).astype(np.float32)
    recall=recall_score(ground_truth[0,:,:,0], prediction, average='macro',zero_division=1) # to avoid zero division
    Recall_values.append (recall)
    print(recall)


df = pd.DataFrame(Recall_values, columns=["Recall"])
df = df[df.Recall != 1.0]    
boxplot = df.boxplot(grid=False,vert=True,color='blue')
mean_rec = df.mean().values
std=df.std()
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean Recall is: ", mean_rec)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)

Precision for all images

In [ ]:

Precision_values = []
for img in range(a.shape[0]):

    temp_img = a[img]
    ground_truth=b[img]
    temp_img_input=np.expand_dims(temp_img, 0)
    prediction = (model.predict(temp_img_input)[0,0,:,:,0]> 0.5).astype(np.float32)
    precision=precision_score(ground_truth[0,:,:,0], prediction, average='macro',zero_division=1) # to avoid zero division
    Precision_values.append (precision)
    print(precision)


df = pd.DataFrame(Precision_values, columns=["Precision"])
df = df[df.Precision != 1.0]    
mean_precision = df.mean().values
std=df.std()
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean Precision is: ", mean_precision)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)

###Final Metrics excluding black masks and first slices 

Mean IoU

In [ ]:
IoU_values = []
n_pixel_true=[]
n_pixel_predicted=[]
for img in range(70,a.shape[0]): #70 represents the number of slice in which the IoU starts to be high. Here the predicted masks and the real masks are similar. This starting value changes from patient to patient.
    temp_img = a[img]
    ground_truth=b[img]
    if (ground_truth.max()>0):
        temp_img_input=np.expand_dims(temp_img, 0)
        prediction = (model.predict(temp_img_input)[0,0,:,:,0]>0.5).astype(np.uint8) # if there are values greater than 0 , the real mask contains useful information and cannot be discarded
      
        n_pixel_true1=ground_truth.sum() #real area
        n_pixel_predicted1=prediction.sum() #predicted area
        IoU = MeanIoU(num_classes=n_classes)
        IoU.update_state(ground_truth[0,:,:,0], prediction)
        IoU = IoU.result().numpy()
        IoU_values.append(IoU)
        n_pixel_true.append(n_pixel_true1)
        n_pixel_predicted.append(n_pixel_predicted1)



        print(IoU)
  

df = pd.DataFrame(IoU_values, columns=["IoU"])
df = df[df.IoU != 1.0]    
mean_IoU = df.mean().values
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print('Number of masks is', len(IoU_values))
print("Mean IoU is: ", mean_IoU)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)

Binary Accuracy

In [ ]:
BinaryAccuracy_values = []
for img in range(70,a.shape[0]): #70 represents the number of slice in which the IoU starts to be high. Here the predicted masks and the real masks are similar. This starting value changes from patient to patient.
    temp_img = a[img]
    ground_truth=b[img]
    if (ground_truth.max()>0):
        temp_img_input=np.expand_dims(temp_img, 0)
        prediction = (model.predict(temp_img_input)[0,0,:,:,0]>0.5).astype(np.uint8) # if there are values greater than 0 , the real mask contains useful information and cannot be discarded


        Accuracy=BinaryAccuracy()
        Accuracy.update_state(ground_truth[:,:,0], prediction)
        Accuracy = Accuracy.result().numpy()
        BinaryAccuracy_values.append(Accuracy)

        print(Accuracy)
      

df = pd.DataFrame(BinaryAccuracy_values, columns=["BinaryAccuracy"])
df = df[df.BinaryAccuracy != 1.0]    
mean_acc = df.mean().values
std=df.std()
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean Accuracy is: ", mean_acc)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)

Recall

In [ ]:

Recall_values = []
for img in range(70,a.shape[0]): #70 represents the number of slice in which the IoU starts to be high. Here the predicted masks and the real masks are similar. This starting value changes from patient to patient.
    temp_img = a[img]
    ground_truth=b[img]
    if (ground_truth.max()>0):
        temp_img_input=np.expand_dims(temp_img, 0)
        prediction = (model.predict(temp_img_input)[0,0,:,:,0]>0.5).astype(np.uint8) # if there are values greater than 0 , the real mask contains useful information and cannot be discarded

        recall=recall_score(ground_truth[0,:,:,0], prediction, average='macro',zero_division=1)
        Recall_values.append (recall)
        print(recall)


df = pd.DataFrame(Recall_values, columns=["Recall"])
df = df[df.Recall != 1.0]    
boxplot = df.boxplot(grid=False,vert=True,color='blue')
mean_rec = df.mean().values
std=df.std()
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean Recall is: ", mean_rec)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)

Precision

In [ ]:

Precision_values = []
for img in range(70,a.shape[0]): #70 represents the number of slice in which the IoU starts to be high. Here the predicted masks and the real masks are similar. This starting value changes from patient to patient.
    temp_img = a[img]
    ground_truth=b[img]
    if (ground_truth.max()>0):
        temp_img_input=np.expand_dims(temp_img, 0)
        prediction = (model.predict(temp_img_input)[0,0,:,:,0]>0.5).astype(np.uint8) # if there are values greater than 0 , the real mask contains useful information and cannot be discarded


        precision=precision_score(ground_truth[0,:,:,0], prediction, average='macro',zero_division=1)
        Precision_values.append (precision)
        print(precision)


df = pd.DataFrame(Precision_values, columns=["Precision"])
df = df[df.Precision != 1.0]    
mean_precision = df.mean().values
std=df.std()
boxplot = df.boxplot(grid=False,vert=True,color='r') #Box Plot
std=df.std() # Standard Deviation
median=df.median() #Median

q1=df.quantile(0.25)
q3= df.quantile(0.75)
iqr=q3-q1 #InterQuartile Range
print("Mean Precision is: ", mean_precision)
print("standard deviation is ",std)
print('median is ',median)
print('iqr is ',iqr)